In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 6, 26)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-06-26
yesterday: 2023-06-23


### Tables in the process

In [3]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [4]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,CPNREIT,129,2023-06-20,"3,000",10.20,1034,1266,2023-06-23,10.70,66.45,4.65,"32,028.90",3,"1,361.13",4.44%,539.99%,-34,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [5]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)
name

'CPNREIT'

In [6]:
sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1266



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,CPNREIT,1266,129,2023-06-20,"3,000",10.20,63.34,4.43,Sold,"30,667.77",HD,None


In [7]:
transaction, name, qty, sell_price, buy_price

('S', 'CPNREIT', -3000, 10.7, 10.2)

### In case of buy, retreive buy id from latest buy record (id)

In [5]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,EA,1269,38,2023-06-26,400,53.00,43.88,3.07,Active,"21,246.95",DTD,None


In [6]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'EA', 400, 53.0)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [8]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'CPNREIT'



,name,date,shares,price,active,period,grade
0,CPNREIT,2022-08-16,"63,000",17.20,1,2,A2


In [9]:
new_cost = 17.55
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + -3000, price = 17.55
WHERE name = 'CPNREIT'



In [10]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [11]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'CPNREIT'



,name,date,shares,price,active,period,grade
0,CPNREIT,2022-08-16,"60,000",17.55,1,2,A2


### If shares become zero, must delete from buy table

In [13]:
sqlDel = """
DELETE FROM buy
WHERE name = '%s' AND volbuy = 0"""
sqlDel = sqlDel % name
print(sqlDel)


DELETE FROM buy
WHERE name = 'TFFIF' AND volbuy = 0


In [14]:
rp = const.execute(sqlDel)
rp.rowcount

1

### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [12]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'CPNREIT'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,CPNREIT,0.0607,0.3000,0.2800,0.2470,0.8877,"63,000",2023-05-23,2023-06-08,,1


In [13]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + -3000
WHERE name = 'CPNREIT'



In [14]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [15]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'CPNREIT'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,CPNREIT,0.0607,0.3000,0.2800,0.2470,0.8877,"60,000",2023-05-23,2023-06-08,,1


### If shares become zero and want to delete from DIVIDEND table

In [ ]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s'
"""
sqlDel = sqlDel % (name)
print(sqlDel)

In [ ]:
rp = const.execute(sqlDel)
rp.rowcount

### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [19]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'TFFIF'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9739,TFFIF,8.50,7.00,S,7.65,7.70,24.39,0.41,7.65,"20,000",-4,4,"20,000",0,0,DOS,SET


In [20]:
buy_target  = 7.65 # must calculate manually beforehand
sell_target = 7.70
cost = 7.65
status = 'O'
reason = "DOS"
sell_qty = 10000

sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s,
qty = %s, buy_target = %s, sell_target = %s, cost = %s, reason = '%s', status = '%s'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, sell_qty, buy_target, sell_target, cost, reason, status, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + -20000,
qty = 10000, buy_target = 7.65, sell_target = 7.7, cost = 7.65, reason = 'DOS', status = 'O'
WHERE name = 'TFFIF'



In [21]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [22]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'TFFIF'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9739,TFFIF,8.50,7.00,O,7.65,7.70,24.39,0.41,7.65,"10,000",-4,4,0,0,0,DOS,SET


### End of Update stocks table in SQLite port_lite database from sale transaction

### If shares become zero and no longer wants to keep STOCK record

In [ ]:
sqlDel = """
DELETE FROM stocks
WHERE name = '%s' AND available_qty <= 0
"""
sqlDel = sqlDel % name
print(sqlDel)

In [ ]:
rp = conlite.execute(sqlDel)
rp.rowcount

## End of Sale process

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [7]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy['date'] = pd.to_datetime(buy['date'])
buy.drop(['volsell', 'volbal'], axis=1, inplace = True)
buy.rename(columns={'volbuy':'shares'}, inplace = True)
buy['shares'] = buy['shares'].astype('int64')
buy


SELECT * 
FROM buy 
WHERE name = 'EA'



,name,date,shares,price,active,dividend,period,grade


### In case there is the same stock name in port

In [22]:
qty = 1200
new_cost = 29.00 # new average cost must be calculated manually beforehand
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + 1200, price = 29.0
WHERE name = 'JMART'



In [23]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [24]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'JMART'



,name,date,shares,price,active,dividend,period,grade
0,JMART,2023-01-20,"9,000",29.00,1,1.0600,4,C3


### In case of totally new stock

In [8]:
rcd = ['EA', '2023-06-26', 400, 53, 0, 0, 1, 0, 4,'A3']
rcd

['EA', '2023-06-26', 400, 53, 0, 0, 1, 0, 4, 'A3']

In [9]:
sqlIns = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [10]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

In [11]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'EA'



,name,date,shares,price,active,dividend,period,grade
0,EA,2023-06-26,400,53.00,1,0.0000,4,A3


### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [25]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'JMART' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,JMART,0.6600,0.0000,0.4000,0.0000,1.0600,"7,800",2023-04-11,2023-05-03,,0


In [26]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 1200
WHERE name = 'JMART'



In [27]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [28]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'JMART' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,JMART,0.6600,0.0000,0.4000,0.0000,1.0600,"9,000",2023-04-11,2023-05-03,,0


### In case of totally new stock

In [ ]:
rcd = ['ASK', 1.44, 0, 0, 0, 1.44, 4500, '2023-02-28', '2023-04-27', '', 1]
rcd

In [ ]:
sqlIns = """
INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)

In [ ]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

### End of Create dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [12]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks


SELECT * 
FROM stocks 
WHERE name = 'EA'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9753,EA,0,0,O,61,64,0,0,61,400,-4,4,0,0,0,52WL,SET50


In [13]:
new_cost = 53
buy_target = 53
sell_target = 55.75
status = 'I'
reason = '5pct'
new_buy_qty = 400
new_sell_qty = 400
new_cost, buy_target, sell_target, status, reason, new_buy_qty, new_sell_qty 

(53, 53, 55.75, 'I', '5pct', 400, 400)

In [14]:
#qty = 12000
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, 
cost = %s, buy_target = %s, sell_target = %s, status = '%s', 
reason = '%s', qty = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, buy_target, sell_target, status, reason, new_sell_qty, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 400, 
cost = 53, buy_target = 53, sell_target = 55.75, status = 'I', 
reason = '5pct', qty = 400
WHERE name = 'EA'



In [15]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [16]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'EA'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9753,EA,0.00,0.00,I,53.00,55.75,0.00,0.00,53.00,400,-4,4,400,0,0,5pct,SET50


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process

In [17]:
sql = "SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '%s' \
ORDER BY period, buy.name"
sql = sql % (yesterday)
print(sql)

SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '2023-06-23' ORDER BY period, buy.name


In [18]:
output = pd.read_sql(sql, const)
output

,period,grade,name,date,volbuy,buy_price,mkt_price,amtbuy,amtmkt,amtpol,pctpol
0,1,C1,JASIF,2018-05-17,"130,000",10.00,7.10,"1,300,000.00","923,000.00","-377,000.00",-29.00
1,1,C4,MCS,2016-09-20,"78,000",15.10,6.75,"1,177,800.00","526,500.00","-651,300.00",-55.30
2,1,C1,PTTGC,2021-03-17,"6,000",64.75,37.25,"388,500.00","223,500.00","-165,000.00",-42.47
3,1,C2,TMT,2021-08-16,"36,000",10.20,7.35,"367,200.00","264,600.00","-102,600.00",-27.94
4,2,A4,ASK,2023-02-16,"4,500",31.00,25.00,"139,500.00","112,500.00","-27,000.00",-19.35
5,2,C4,ASP,2022-03-07,"30,000",3.80,2.82,"114,000.00","84,600.00","-29,400.00",-25.79
6,2,A3,BANPU,2023-01-06,"30,000",11.50,9.15,"345,000.00","274,500.00","-70,500.00",-20.43
7,2,A2,CPNREIT,2022-08-16,"60,000",17.55,11.00,"1,053,000.00","660,000.00","-393,000.00",-37.32
8,2,C1,DIF,2020-08-01,"32,000",14.45,10.60,"462,400.00","339,200.00","-123,200.00",-26.64
9,2,B1,GVREIT,2022-08-24,"30,000",8.90,8.45,"267,000.00","253,500.00","-13,500.00",-5.06


In [19]:
file_name = 'portfolio.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

output.to_csv(output_file, header=True, index=False)
output.to_csv(data_file, header=True, index=False)
output.to_csv(one_file, header=True, index=False)
output.to_csv(osd_file, header=True, index=False)